## Getting fundamental data
The fundamentals that I want are:
* Easy to get:
    * Company headquarters location
    * Sector
    * Outstanding shares (for market cap, want weekly)
    * Earnings yield (inverse of P/E)
    * Cash & equivalents (for dilution prediction)
    * Cash burn (for dilution prediction, but must be estimated)

* Hard to get:
    * Free float (outstanding - insider - institutional)

* Very hard to get:
    * Short interest

A list of what I have found:
* [Polygon](https://polygon.io/docs/stocks/get_vx_reference_financials): only has EPS, sector and outstanding shares
* [Finnhub](https://finnhub.io/docs/api): has all easy to get data, but not for delisted stocks
* [FinancialModelingPrep](https://site.financialmodelingprep.com/): has all easy to get. Free float can be calculated as they have insider/institutional holdings and outstanding shares. Delisted tickers only available if ticker is not recycled.
* [Valueinvesting.io](https://valueinvesting.io/short-interest-api): only has short-interest but nothing else, Either 100 or 400/month.
* Compustat: I have access through my university, but not sure if short-interest is available
* [Sharadar](https://data.nasdaq.com/search?filters=%5B%22Equities%22%2C%22Fundamentals%22%5D): Apparently that is the golden standard. Also QuantRocket uses it.
* [Tiingo](https://www.tiingo.com/documentation/)
* [IEXCloud](https://iexcloud.io/)
* [EODHD](https://eodhd.com/pricing?utm_source=google_search&utm_medium=cpc&utm_campaign=Brand_Europe&roistat_referrer&roistat_pos&roistat=google1_g_124657464434_526148784701_eodhd&gad=1)

Since Sharadar is the golden standard and has almost everything I want, including data for ADRs, I settle with them. I thought they are extremely expensive so I didn't look further. Turns out I am wrong, but it's still stupid you have to create an account just to see the pricing. The bundle for all data is 69/month. And they have a bulk data download endpoint. They do not have the headquarters location, so I will just drop that requirement. I can just compare ADRs with non-ADRs after all. Although they do not provide float, it can be calculated by subtracting the institutional and insider ownership from the outstanding shares. While this is not 100% correct (e.g. you also have to take into account options, Rule 144 etc.) it is good enough for my purposes.

Almost no one has short interest. I will likely settle with Compustat and Valueinvesting.io for that. However this is not high on my priorities list.

Other alternative data that I want are the historical borrowing rates on IBKR. These can be bought on [iborrowdesk.com](https://www.patreon.com/iborrowdesk) for almost free.


In [2]:
from datetime import datetime, date, time, timedelta
from utils import first_trading_day_after, last_trading_day_before, get_tickers_v3
import pandas as pd
import numpy as np

DATA_PATH = "../../../data/polygon/"

START_DATE = first_trading_day_after(datetime(2021, 1, 1).date())
END_DATE = last_trading_day_before(datetime(2023, 8, 18).date())
print(START_DATE)
print(END_DATE)

with open(DATA_PATH + "secret.txt") as f:
    KEY = next(f).strip()

client = RESTClient(api_key=KEY)

2021-01-04
2023-08-18


Polygon actually does not have all tickers. They probably don't have ADRs because their filings are a little bit different.

In [22]:
data = pd.DataFrame(client.vx.list_stock_financials(ticker = "TOP", filing_date_gte=START_DATE, filing_date_lte=END_DATE) )
print(data)
data = client.get_ticker_details(ticker="TBIO", date=END_DATE)
print(data)

Empty DataFrame
Columns: []
Index: []
TickerDetails(active=True, address=CompanyAddress(address1='10431 WATERIDGE CIRCLE', address2='SUITE 150', city='SAN DIEGO', state='CA', country=None, postal_code='92121'), branding=Branding(icon_url='https://api.polygon.io/v1/reference/company-branding/d3d3LnRlbGVzaXNiaW8uY29t/images/2023-05-01_icon.jpeg', logo_url='https://api.polygon.io/v1/reference/company-branding/d3d3LnRlbGVzaXNiaW8uY29t/images/2023-05-01_logo.svg', accent_color=None, light_color=None, dark_color=None), cik='0001850079', composite_figi='BBG011363VM3', currency_name='usd', currency_symbol=None, base_currency_name=None, base_currency_symbol=None, delisted_utc=None, description='Telesis Bio Inc is a synthetic biology company focused on enabling researchers to rapidly, accurately, and reproducibly build or write high-quality synthetic DNA and mRNA that is ready to use in many downstream synthetic biology-enabled markets. It manufactures and sells laboratory equipment, specificall